In [1]:
# | default_exp classes.DomoApplication

In [2]:
# | exporti
import pandas as pd
from dataclasses import dataclass, field
from typing import Optional, List
import httpx

from nbdev.showdoc import patch_to

import domolibrary.routes.application as application_routes
import domolibrary.classes.DomoApplication_Job as dmdj
import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda

In [3]:
# | hide
import os
from pprint import pprint

# What are applications?
Applications tend to be tooling that domo implemented on the executor framework or extensions to domo products that originated as custom apps that have been fully integrated into the UI.

For example Publish started out as a 3rd party app that was later integrated into domo as an App, and then later received dedicated API paths.

Similarly Remote DomoStats / Observability metrics started out as 3rd party scripting, which was later integrated into Domo's "Executor Framework" an internal framework for managing java / lambda functions, now defined under the application APIs.

Note, pieces of the executor framework has slowly been morphed and adapted into CodeEngine and exposed to end users.

In [4]:
from enum import Enum
import domolibrary.utils.convert as cc

In [5]:
class DomoJob_Types(Enum):
    REMOTE_DOMO_STATS = dmdj.DomoJob_RemoteDomoStats
    DATA_WATCHDOG = dmdj.DomoJob_Watchdog

    default = dmdj.DomoJob

    @staticmethod
    def _convert_api_name_to_member_name(api_name):
        return cc.convert_str_to_snake_case(api_name, is_only_alphanumeric=True).upper().replace("TOOLKIT_", '')
    
    @classmethod
    def get_from_api_name(cls, api_name):
        member_name = cls._convert_api_name_to_member_name(api_name)

        if member_name not in cls.__members__:
            return cls['default'].value
        
        return cls[member_name].value

DomoJob_Types.get_from_api_name('Toolkit: Remote Domo Stat')

domolibrary.classes.DomoApplication_Job.DomoJob

In [6]:
# token_auth = dmda.DomoTokenAuth(
#     domo_instance="domo-community",
#     domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
# )

# res = await application_routes.get_applications(auth = token_auth)

# [app.get('name') for app in res.response]


In [7]:
# | exports


@dataclass
class DomoApplication:
    auth: dmda.DomoAuth = field(repr=False)
    id: str
    version : str = None
    name: str = None
    customer_id: str = None
    description: str = None
    execution_class: str = None
    grants: List[str] = None
    jobs: List[dmdj.DomoJob] = field(default=None)
    jobs_schedule: List[dmdj.DomoTrigger_Schedule] = field(default=None, repr=False)


    @classmethod
    def _from_json(cls, obj, auth: dmda.DomoFullAuth = None):
        dd = util_dd.DictDot(obj)

        return cls(
            id=dd.applicationId,
            customer_id=dd.customerId,
            name=dd.name,
            description=dd.description,
            version=dd.version,
            execution_class=dd.executionClass,
            grants=dd.authorities,
            auth=auth
        )
    
    def _get_job_class(self):
        return DomoJob_Types.get_from_api_name(self.name)

In [8]:
# | exporti
@patch_to(DomoApplication, cls_method=True)
async def get_from_id(
    cls,
    auth: dmda.DomoAuth,
    application_id,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=2,
):
    res = await application_routes.get_application_by_id(
        application_id=application_id,
        auth=auth,
        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    return cls._from_json(obj=res.response, auth=auth)

In [9]:
# | eval : False
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-alpha",
    domo_access_token=os.environ["ALPHA_ACCESS_TOKEN"],
)
test_application = await DomoApplication.get_from_id(
    auth=token_auth,
    application_id="33aab8f0-3397-45e1-933c-755abd0f5b3a",
    return_raw=False,
)

pprint(test_application)

warning this token has not been validated by who_am_i, run get_auth_token first
DomoApplication(id='33aab8f0-3397-45e1-933c-755abd0f5b3a',
                version='1.0.46_2',
                name='Toolkit: Data Watchdog',
                customer_id='domo-alpha',
                description='Monitors DataSets and DataFlows for execution '
                            'errors or execution duration outliers.',
                execution_class='com.domo.executor.datawatchdog.DataWatchdogExecutor',
                grants=['dataflow.manage',
                        'pipeline.executor.job.create',
                        'dataset.manage'],
                jobs=None)


In [10]:
@patch_to(DomoApplication)
async def get_jobs(
    self,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
    return_raw: bool = False,
):

    res = await application_routes.get_application_jobs(
        auth=self.auth,
        application_id=self.id,
        debug_api=debug_api,
        session=session,
        parent_class=self.__class__.__name__,
    )

    if return_raw:
        return res
    
    job_cls = self._get_job_class()

    self.jobs = [job_cls._from_json(job, auth=self.auth) for job in res.response]
    return self.jobs

In [11]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

test_application = await DomoApplication.get_from_id(
    auth=token_auth, application_id="50e7230f-d2f2-42e2-a208-d94c8ae9f64c"
)

pprint(await test_application.get_jobs())

warning this token has not been validated by who_am_i, run get_auth_token first
[DomoJob_RemoteDomoStats(name='domo-community',
                         application_id='50e7230f-d2f2-42e2-a208-d94c8ae9f64c',
                         logs_dataset_id='da68b3ea-088f-4493-ae69-ee5094feaa29',
                         id='f7a1b31f-f774-4eeb-ba85-d56d29b1dedc',
                         user_id=1893952720,
                         execution_timeout=1440,
                         is_enabled=True,
                         customer_id='mmmm-0012-0200',
                         created_dt=datetime.datetime(2024, 2, 26, 17, 10, 31),
                         updated_dt=datetime.datetime(2024, 2, 27, 11, 13, 31),
                         description='updated on 2024-02-27',
                         execution_payload={'metricsDatasetId': 'da68b3ea-088f-4493-ae69-ee5094feaa29',
                                            'policies': {'AccountPermissions': 'b156bfe4-d0e5-4410-8354-b3062973a99c'},
      

In [12]:
@patch_to(DomoApplication)
async def get_schedules(self) -> pd.DataFrame:
    if not self.jobs:
        await self.get_jobs()

    # return self.jobs

    schedules = pd.DataFrame(
        [
            {
                **trigger.schedule.to_obj(),
                "job_name": job.name,
                "job_id": job.id,
                "description": job.description,
                "remote_instance" : job.remote_instance,
                "application" : self.name
            }
            for job in self.jobs
            for trigger in job.triggers
        ]
    )

    # return schedules

    self.jobs_schedule = schedules.sort_values(
        ["hour", "minute"], ascending=True
    ).reset_index(drop=True)
    return self.jobs_schedule

In [13]:
#| eval : false

test_application = await DomoApplication.get_from_id(
    auth=token_auth, application_id="50e7230f-d2f2-42e2-a208-d94c8ae9f64c"
)

await test_application.get_schedules()

,hour,minute,job_name,job_id,description,remote_instance,application
0,1,29,domo-community,f7a1b31f-f774-4eeb-ba85-d56d29b1dedc,updated on 2024-02-27,domo-community,Toolkit: Remote Domo Stats


In [14]:
# | exporti
@patch_to(DomoApplication)
async def find_next_job_schedule(self, return_raw: bool = False) -> dmdj.DomoTrigger_Schedule:

    await self.get_jobs()
    await self.get_schedules()

    df_all_hours = pd.DataFrame(range(0, 23), columns=["hour"])
    df_all_minutes = pd.DataFrame(range(0, 60), columns=["minute"])

    df_all_hours["tmp"] = 1
    df_all_minutes["tmp"] = 1
    df_all = pd.merge(df_all_hours, df_all_minutes, on="tmp").drop(columns=["tmp"])

    # get the number of occurencies of each hour and minutes
    schedules_grouped = (
        self.jobs_schedule.groupby(["hour", "minute"])
        .size()
        .reset_index(name="cnt_schedule")
    )

    # print(schedules_grouped)
    # print(df_all)

    schedules_interpolated = pd.merge(
        df_all, schedules_grouped, how="left", on=["hour", "minute"]
    )

    schedules_interpolated["cnt_schedule"] = schedules_interpolated[
        "cnt_schedule"
    ].fillna(value=0)
    schedules_interpolated.sort_values(
        ["cnt_schedule", "hour", "minute"], ascending=True, inplace=True
    )

    schedules_interpolated.reset_index(drop=True, inplace=True)

    if return_raw:
        return schedules_interpolated

    return dmdj.DomoTrigger_Schedule(
        hour=int(schedules_interpolated.loc[0].get("hour")),
        minute=int(schedules_interpolated.loc[0].get("minute")),
    )

In [15]:
#| eval : false
test_application = await DomoApplication.get_from_id(
    auth=token_auth, application_id="50e7230f-d2f2-42e2-a208-d94c8ae9f64c"
)
await test_application.find_next_job_schedule(return_raw= False)



DomoTrigger_Schedule(schedule_text=None, schedule_type='scheduleTriggered', minute=0, hour=0, minute_str=None, hour_str=None)

In [16]:
#| hide
import nbdev
nbdev.nbdev_export('50_DomoApplication.ipynb')